In [0]:
%sql
CREATE or REPLACE TABLE  employees (
    emp_id INT,
    emp_name STRING,
    dept_id INT,
    salary INT,
    manager_id INT,
    hire_date DATE
)USING DELTA;

In [0]:
%sql
INSERT INTO employees VALUES
(1, 'Alice', 10, 90000, NULL, '2020-01-15'),
(2, 'Bob', 10, 75000, 1, '2021-03-10'),
(3, 'Charlie', 10, 60000, 1, '2022-07-20'),
(4, 'David', 20, 85000, NULL, '2019-11-05'),
(5, 'Eva', 20, 70000, 4, '2021-06-18'),
(6, 'Frank', 20, NULL, 4, '2023-02-01'),       -- NULL salary
(7, 'Grace', 30, 95000, NULL, '2018-09-12'),
(8, 'Hank', 30, 80000, 7, '2020-12-25'),
(9, 'Ivy', 30, 80000, 7, '2021-12-25'),        -- Duplicate salary
(10, 'Jack', 30, 72000, 7, '2022-05-30'),
(11, 'Bob', 10, 75000, 1, '2023-08-15');       -- Duplicate name


In [0]:
%sql
select * from employees

In [0]:
%sql
/*Q1: Remove duplicates based on emp_name (keep latest hire_date)*/
with aa as (
select *,row_number() over (partition by emp_name order by hire_date desc) as rnk  from employees )
select * from aa where rnk=1




In [0]:
%sql
/*Q2: Replace NULL salary with department average*/
WITH aa
     AS (SELECT *,
                Avg(salary)
                  OVER(
                    partition BY dept_id) AS avg_sal
         FROM   employees)
SELECT emp_id,
       emp_name,
       dept_id,
       salary,
       avg_sal,
       CASE
         WHEN salary IS NULL THEN avg_sal
         ELSE salary
       END AS new_salary,
       coalesce(salary,avg_sal) as avg_sal1 
FROM   aa 




In [0]:
%sql
/*Q3: Find top 3 earners per department*/
with aa as (
select *,row_number() over(partition by dept_id order by salary desc ) as rnk from employees)
select * from aa where rnk<=3

In [0]:
%sql
/*Q4: Flag employees above department average*/
with aa as (
select *, avg(salary) over(partition by dept_id ) as avg_sal from employees)
select *, case when salary > avg_sal then 'abve_avg_sal' else 'low_avg_sal' end as sal_flg from aa




In [0]:
%sql
/*Q5: Compare employee salary vs manager salary*/
select e.emp_id,e.emp_name,e.manager_id,e.salary,m.emp_id as manager_id,m.emp_name as manager , m.salary, (m.salary-e.salary) as sal_diff from employees e left join employees m on e.manager_id=m.emp_id --where e.salary>m.salary

In [0]:
%sql
/*Q6: Find duplicate salaries*/
select salary,count(*) as cnt from employees group by salary having cnt>1


In [0]:
%sql
/*Q7: Find nth highest salary (e.g., 2nd highest)*/
with aa as (
select *,row_number() over(order by salary desc) as rnk from employees)
select * from aa where rnk=2


In [0]:
%sql
/*Q8: Running total of salary per department*/
select *,avg(salary) over (partition by dept_id order by salary) as avg_sal from employees

. Advanced Questions & Answers
Q1: Remove duplicates based on emp_name (keep latest hire_date)
SQLSELECT emp_id, emp_name, dept_id, salary, hire_dateFROM (    SELECT *,           ROW_NUMBER() OVER (PARTITION BY emp_name ORDER BY hire_date DESC) AS rn    FROM employees) tWHERE rn = 1;Show more lines

Q2: Replace NULL salary with department average
SQLSELECT emp_id, emp_name, dept_id,       COALESCE(salary, avg_sal) AS final_salaryFROM (    SELECT *,           AVG(salary) OVER (PARTITION BY dept_id) AS avg_sal    FROM employees) t;Show more lines

Q3: Find top 3 earners per department
SQLSELECT emp_id, emp_name, dept_id, salaryFROM (    SELECT *,           ROW_NUMBER() OVER (PARTITION BY dept_id ORDER BY salary DESC) AS rn    FROM employees) tWHERE rn <= 3;Show more lines

Q4: Flag employees above department average
SQLSELECT emp_id, emp_name, dept_id, salary,       CASE WHEN salary > AVG(salary) OVER (PARTITION BY dept_id)            THEN 'high_sal' ELSE 'low_sal' END AS sal_grpFROM employees;Show more lines

Q5: Compare employee salary vs manager salary
SQLSELECT e.emp_name AS employee,       e.salary AS emp_salary,       m.emp_name AS manager,       m.salary AS mgr_salary,       (e.salary - m.salary) AS diffFROM employees eLEFT JOIN employees m ON e.manager_id = m.emp_id;Show more lines

Q6: Find duplicate salaries
SQLSELECT salary, COUNT(*) AS cntFROM employeesGROUP BY salaryHAVING COUNT(*) > 1;Show more lines

Q7: Find nth highest salary (e.g., 2nd highest)
SQLSELECT DISTINCT salaryFROM employeesORDER BY salary DESCLIMIT 1 OFFSET 1;Show more lines

Q8: Running total of salary per department
SQLSELECT emp_id, emp_name, dept_id, salary,       SUM(salary) OVER (PARTITION BY dept_id ORDER BY hire_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_totalFROM employees;Show more lines

Q9: Calculate cumulative percentage of salary per department
SQLSELECT emp_id, emp_name, dept_id, salary,       SUM(salary) OVER (PARTITION BY dept_id ORDER BY salary DESC) /       SUM(salary) OVER (PARTITION BY dept_id) * 100 AS cumulative_pctFROM employees;Show more lines

Q10: Find employees hired in last 1 year
SQLSELECT *FROM employeesWHERE hire_date >= DATE_SUB(CURRENT_DATE(), 365);Show more lines

Q11: Use NTILE to divide employees into 4 salary bands
SQLSELECT emp_id, emp_name, dept_id, salary,       NTILE(4) OVER (ORDER BY salary DESC) AS salary_bandFROM employees;Show more lines

Q12: Pivot department-wise total salary
SQLSELECT *FROM (    SELECT dept_id, salary FROM employees) srcPIVOT (    SUM(salary) FOR dept_id IN (10, 20, 30));Show more lines

Q13: Find employees with same salary as their manager
SQLSELECT e.emp_name, e.salary, m.emp_name AS managerFROM employees eJOIN employees m ON e.manager_id = m.emp_idWHERE e.salary = m.salary;Show more lines

Q14: Find employees with duplicate names
SQLSELECT emp_name, COUNT(*) AS cntFROM employeesGROUP BY emp_nameHAVING COUNT(*) > 1;Show more lines

Q15: Find employees earning more than department median
(Databricks supports percentile_approx)
SQLSELECT emp_id, emp_name, dept_id, salaryFROM employeesWHERE salary > (    SELECT percentile_approx(salary, 0.5)    FROM employees e2    WHERE e2.dept_id = employees.dept_id);Show more lines

Q16: Show previous employee salary using LAG
SQLSELECT emp_id, emp_name, salary,       LAG(salary) OVER (ORDER BY hire_date) AS prev_salaryFROM employees;Show more lines

Q17: Show next employee salary using LEAD
SQLSELECT emp_id, emp_name, salary,       LEAD(salary) OVER (ORDER BY hire_date) AS next_salaryFROM employees;Show more lines

Q18: Find department with highest total salary
SQLSELECT dept_id, SUM(salary) AS total_salaryFROM employeesGROUP BY dept_idORDER BY total_salary DESCLIMIT 1;Show more lines

Q19: Find employees hired before their manager
SQLSELECT e.emp_name, e.hire_date, m.emp_name AS manager, m.hire_date AS mgr_hire_dateFROM employees eJOIN employees m ON e.manager_id = m.emp_idWHERE e.hire_date < m.hire_date;Show more lines

Q20: Count employees per department and rank departments
SQLSELECT dept_id, COUNT(*) AS emp_count,       RANK() OVER (ORDER BY COUNT(*) DESC) AS dept_rankFROM employeesGROUP BY dept_id;

In [0]:
%sql
select e.emp_id,e.emp_name,m.emp_name as manager_name from employees e  left join employees m on e.manager_id=m.emp_id

In [0]:
%sql
with aa as (
 select *,row_number() over(order by salary desc ) as rn from employees order by salary desc )

 select emp_id,emp_name,dept_id,salary from aa where rn=2

In [0]:
%sql
SELECT *,
       AVG(salary) OVER (PARTITION BY dept_id) AS avg_sal,
       CASE 
           WHEN salary > AVG(salary) OVER (PARTITION BY dept_id) THEN 'high_sal'
           ELSE 'low_sal'
       END AS sal_grp
FROM employees;